In [1]:
#Import dependencies
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup as bs
import requests as req
from splinter import Browser

## Mars News

In [2]:
#Scrape NASA site
url = "https://mars.nasa.gov/news"
browser = Browser('chrome', headless=False)
browser.visit(url)
response = req.get(url)
soup = bs(response.text, 'lxml')

In [3]:
#Grab and assign variables to the most recent article title and paragraph
news_title = soup.find("div", class_="content_title").text
news_paragraph = soup.find("div", class_="rollover_description_inner").text

In [5]:
#Print the title and paragraph to doublecheck against the URL
print(news_title)
print(news_paragraph)



NASA Invests in Visionary Technology 



NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.



## JPL Mars Space Images - Featured Image

In [6]:
#Visit the url for JPL's Featured Space Image
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser = Browser('chrome', headless=False)
browser.visit(url)

In [7]:
#Use Splinter API to click through to the featured image
browser.click_link_by_id('full_image')

In [8]:
#Click on More Info
browser.click_link_by_partial_text('more info')

In [9]:
#Parse through the page to find the link to the high res jpg
html = browser.html
soup = bs(html, 'html.parser')
url = soup.find('img', class_='main_image')
image = url.get('src')

In [10]:
#Set variable to complete image url
featured_image_url = "https://www.jpl.nasa.gov" + image

In [11]:
#Print the link to double-check against original link
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18791_hires.jpg


## Mars Weather

In [12]:
#Scrape the Mars Twitter page 
twitter_response = req.get("https://twitter.com/marswxreport?lang=en")
twitter_soup = bs(twitter_response.text, 'html.parser')
tweets = twitter_soup.find_all('div', class_="js-tweet-text-container")

In [13]:
#Print most recent post from Twitter feed
mars_weather = tweets[0].text
print(mars_weather)


Sol 2017 (April 09, 2018), Sunny, high -6C/21F, low -75C/-103F, pressure at 7.17 hPa, daylight 05:28-17:21



## Mars Facts

In [14]:
#Access Mars Facts page and scrape table
mars_facts = req.get("https://space-facts.com/mars/")
read_facts = pd.read_html(mars_facts.text)

In [15]:
#Print out table results
read_facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [16]:
#Use pandas to convert info into dataframe
mars_df = read_facts[0]
mars_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
#Convert dataframe into html string
mars_html = mars_df.to_html('mars.html')

## Mars Hemispheres

In [18]:
#Visit the USGS Astrogeology site
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html
soup = bs(html, "html.parser")

In [19]:
#Locate four instances of Mars' hemispheres
products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")
mars_hemisphere = []

In [20]:
#Save image URLs in string w/ hemisphere names; store data using keys img_url and title
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    mars_hemisphere.append({"title": title, "img_url": image_url})

In [21]:
#Print strings
mars_hemisphere 

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere '}]